In [1]:
import sys
print(sys.executable)

/home/kwj/team_proj/faiss_2/bin/python3


In [2]:
import sys
print(sys.version)

3.8.20 (default, Sep  7 2024, 18:35:08) 
[GCC 11.4.0]


In [3]:
# !pip install fitz frontend pymupdf

In [4]:
# !pip install pypdf

In [5]:
# !pip install numpy dotenv langchain torch langchain_community transformers datasets

In [6]:
# !pip install --upgrade pybind11
# !pip uninstall faiss-gpu
# !pip install faiss-gpu

In [7]:
# !pip uninstall numpy
# !pip install "numpy<2"

In [1]:
import numpy as np
print(np.__version__)

1.24.4


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

# path = "C:\\KIMUJUNG\\team_project\\team_project3\\pdf_folder"
path = "pdf_folder_hana"
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

file_list: ['21_34_(무)하나 가득담은 3.5.5 간편 건강보험(2411)(간편심사형) 1종(갱신형, 표준형)_pdf.pdf', '2_5_하나업무용자동차보험(다이렉트)_pdf.pdf', '3_2_하나영업용자동차보험_pdf.pdf', '14_3_하나 사이버금융범죄보상보험(Ⅰ)_pdf.pdf', '21_17_(무)하나 가득담은 3.0.5 간편 건강보험(일반심사형)(3대질병형)_pdf.pdf', '12_3_하나 Grade 주택화재보험_pdf.pdf', '11_11_하나 해외여행보험(방카)_pdf.pdf', '21_43_(무)하나 가득담은 3대질환보험(2411)(일반심사형) 2종(세만기형,해약환급금미지급형Ⅱ)_pdf.pdf', '12_1_하나 주택화재보험_pdf.pdf', '21_37_(무)하나 가득담은 3.5.5 간편 건강보험(2411)(일반심사형) 1종(갱신형, 표준형)_pdf.pdf', '5_2_원데이취급사원자동차보험_pdf.pdf', '5_4_원데이렌터카법인자동차보험_pdf.pdf', '21_9_(무)하나 가득담은 암보험(2404) 1종(세만기형, 표준형)_pdf.pdf', '21_36_(무)하나 가득담은 3.5.5 간편 건강보험(2411)(간편심사형) 3종(세만기, 해약환급금미지급형Ⅱ)_pdf.pdf', '21_39_(무)하나 가득담은 3.5.5 간편 건강보험(2411)(일반심사형) 3종(세만기, 해약환급금미지급형Ⅱ)_pdf.pdf', '21_20_(무)하나 가득담은 암보험(TM)(2404)(일반심사형)_pdf.pdf', '11_15_원데이 운전자보험(Ⅱ)_pdf.pdf', '11_20_하나 슬기로운자녀생활보험_pdf.pdf', '21_29_(무)뉴 건강하면 더 좋은 하나의 간편 보험(2409) 1종(세만기형, 일반형)_pdf.pdf', '11_10_하나 실손의료보험(해외여행용)_pdf.pdf', '11_12_원데이 레저보험(Ⅱ)_pdf.pdf', '11_17_하나 해외유학_장기체류보험_pdf.pdf', '21_18_(무)하나 가득담은 치아보험

In [4]:
import os
import faiss
import pickle
import numpy as np
import torch
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModel, AutoTokenizer
import fitz  # PyMuPDF

# 1️⃣ GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/home/kwj/team_proj/faiss_2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [5]:
# path = "C:\\KIMUJUNG\\team_project\\team_project3\\pdf_folder"
path = "pdf_folder_hana"
file_list = os.listdir(path)

In [6]:
# 2️⃣ 모델 및 토크나이저 로드 (GPU로 이동)
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 3️⃣ 임베딩 생성 함수
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    # 모든 입력 텐서를 GPU로 이동
    inputs = {key: val.to(device) for key, val in inputs.items()}
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()  # GPU에서 CPU로 변환 후 반환

# 4️⃣ 저장 경로 설정
faiss_index_path = "./faiss_index_hana.bin"
metadata_path = "./documents_hana.pkl"

# 5️⃣ PDF 문서 리스트
# file_list = ["document1.pdf", "document2.pdf"]  # 실제 파일 경로로 변경
# path = "/your/pdf/path"  # 실제 경로로 변경

# 6️⃣ FAISS GPU 인덱스 생성
res = faiss.StandardGpuResources()  # GPU 리소스 할당
index = None

# 7️⃣ 문서 처리
all_documents = []
for file in file_list:
    file_path = os.path.join(path, file)
    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except:
        documents = []
        with fitz.open(file_path) as pdf:
            metadata = pdf.metadata # PDF의 메타데이터 추출
            
            for page_num, page in enumerate(pdf):
                page_text = page.get_text()
                if "MuPDF error:" not in page_text:
                    documents.append(Document(
                        page_content=page_text,
                        metadata={
                            'producer': metadata.get('producer', ''),
                            'creator': metadata.get('creator', ''),
                            'creationdate': metadata.get('creationDate', ''),
                            'title': metadata.get('title', ''),
                            'author': metadata.get('author', ''),
                            'moddate': metadata.get('modDate', ''),
                            'pdfversion': metadata.get('pdfVersion', ''),
                            'source': file_path,
                            'total_pages': pdf.page_count,
                            'page': page_num,
                            'page_label': str(page_num + 1)
                        }
                    ))

    # 8️⃣ 텍스트 정제
    for doc in documents:
        doc.page_content = doc.page_content.replace("\n", " ").replace("  ", " ")
        doc.metadata["source"] = file

    # 9️⃣ 문서 스플리팅
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(documents)

    # 1️⃣0️⃣ 텍스트 추출 및 임베딩 생성
    texts = [doc.page_content for doc in split_documents]
    embeddings = []

    batch_size = 16
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_embeddings = get_embeddings(batch)
        embeddings.extend(batch_embeddings)

    embeddings = np.array(embeddings, dtype=np.float32)
    
    # 1️⃣1️⃣ FAISS GPU 인덱스 생성 (처음이면 초기화)
    if index is None:
        embedding_dim = embeddings.shape[1]
        cpu_index = faiss.IndexFlatL2(embedding_dim)  # CPU 인덱스
        index = faiss.index_cpu_to_gpu(res, 0, cpu_index)  # GPU로 변환
    
    # 1️⃣2️⃣ FAISS 인덱스에 데이터 추가
    index.add(embeddings)
    all_documents.extend(split_documents)

    print(f"✅ {file} 처리 완료")

# 1️⃣3️⃣ FAISS 인덱스 저장
faiss.write_index(faiss.index_gpu_to_cpu(index), faiss_index_path)

# 1️⃣4️⃣ 문서 저장
with open(metadata_path, "wb") as f:
    pickle.dump(all_documents, f)

print("🎉 모든 문서 처리 및 저장 완료!")


✅ 21_34_(무)하나 가득담은 3.5.5 간편 건강보험(2411)(간편심사형) 1종(갱신형, 표준형)_pdf.pdf 처리 완료
✅ 2_5_하나업무용자동차보험(다이렉트)_pdf.pdf 처리 완료
✅ 3_2_하나영업용자동차보험_pdf.pdf 처리 완료
✅ 14_3_하나 사이버금융범죄보상보험(Ⅰ)_pdf.pdf 처리 완료
✅ 21_17_(무)하나 가득담은 3.0.5 간편 건강보험(일반심사형)(3대질병형)_pdf.pdf 처리 완료
✅ 12_3_하나 Grade 주택화재보험_pdf.pdf 처리 완료
✅ 11_11_하나 해외여행보험(방카)_pdf.pdf 처리 완료
✅ 21_43_(무)하나 가득담은 3대질환보험(2411)(일반심사형) 2종(세만기형,해약환급금미지급형Ⅱ)_pdf.pdf 처리 완료
✅ 12_1_하나 주택화재보험_pdf.pdf 처리 완료
✅ 21_37_(무)하나 가득담은 3.5.5 간편 건강보험(2411)(일반심사형) 1종(갱신형, 표준형)_pdf.pdf 처리 완료
✅ 5_2_원데이취급사원자동차보험_pdf.pdf 처리 완료
✅ 5_4_원데이렌터카법인자동차보험_pdf.pdf 처리 완료
✅ 21_9_(무)하나 가득담은 암보험(2404) 1종(세만기형, 표준형)_pdf.pdf 처리 완료
✅ 21_36_(무)하나 가득담은 3.5.5 간편 건강보험(2411)(간편심사형) 3종(세만기, 해약환급금미지급형Ⅱ)_pdf.pdf 처리 완료
✅ 21_39_(무)하나 가득담은 3.5.5 간편 건강보험(2411)(일반심사형) 3종(세만기, 해약환급금미지급형Ⅱ)_pdf.pdf 처리 완료
✅ 21_20_(무)하나 가득담은 암보험(TM)(2404)(일반심사형)_pdf.pdf 처리 완료
✅ 11_15_원데이 운전자보험(Ⅱ)_pdf.pdf 처리 완료
✅ 11_20_하나 슬기로운자녀생활보험_pdf.pdf 처리 완료
✅ 21_29_(무)뉴 건강하면 더 좋은 하나의 간편 보험(2409) 1종(세만기형, 일반형)_pdf.pdf 처리 완료
✅ 11_10_하나 실손의료보험(해외여행용)_pdf.pdf 처